In [13]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ta
from ydata_profiling import ProfileReport
from model_config import Path
import plotly.express as px

In [14]:
# Параметры
data_folder = Path['raw']  # Путь к папке с данными
time_window = 168  # Временное окно в часах (1 неделя)
hours_cnt = 3860
split_cnt = 0.8

In [15]:
train_data = []
test_data = []
final_data = []

In [16]:
# Получаем список всех CSV-файлов в папке
file_list = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

In [17]:
valid_tokens = {}
token_min_dates = []
token_max_dates = []

In [ ]:
i = 0
for file in file_list:
    file_path = os.path.join(data_folder, file)
    df = pd.read_csv(file_path)
    token_name = file.split("_")[0]

    # if len(df) < hours_cnt: #160 дней 
    #     print(f"Токен {token_name} не имеет достаточного количества данных. Пропускаем его. {df.shape}")
    #     continue
    
    valid_tokens[token_name] = i
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    start_date = df['Date'].min()
    end_date = df['Date'].max()
    df = df.set_index('Date')

    # Создаем полный индекс временных меток с шагом 1 час
    full_time_index = pd.date_range(start=start_date, end=end_date, freq='h')
    df = df.reindex(full_time_index)
    df = df.reset_index()
    df.rename(columns={"index":"Date"}, inplace= True)

    token_min_dates.append(df['Date'].min())
    token_max_dates.append(df['Date'].max())
    i+=1

# Определяем общую конечную дату как самую раннюю из максимальных дат всех токенов
common_start_date = max(token_min_dates)
common_end_date = max(token_max_dates)
print(common_start_date, common_end_date)

In [ ]:
valid_tokens

In [ ]:
for file in file_list:
    file_path = os.path.join(data_folder, file)
    token_name = file.split("_")[0]
    if token_name not in valid_tokens:
        continue

    df = pd.read_csv(file_path)
    
    # Обрабатываем дату
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    start_date = df['Date'].min()
    end_date = df['Date'].max()
    df = df.set_index('Date')

    # Создаем полный индекс временных меток с шагом 1 час
    full_time_index = pd.date_range(start=start_date, end=end_date, freq='h')
    df = df.reindex(full_time_index)

    # Заполняем пропущенные значения путем интерполяции
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].interpolate(method='linear')

    # Отбираем данные в диапазоне от common_start_date до common_end_date
    # df = df[(df.index >= common_start_date) & (df.index <= common_end_date)]

    # Нормализуем цену закрытия
    df['Close_Normalized'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()
    
    # Вычисляем доходность
    df['Return'] = df['Close'].pct_change()
    
    # Вычисляем экспоненциально взвешенное стандартное отклонение доходности (волатильность) с окном в 168 часов
    df['Volatility'] = df['Return'].ewm(span=168, adjust=False).std()
    
    # Нормализуем волатильность
    df['Volatility_Normalized'] = df['Volatility'] / df['Volatility'].mean()
    
    # Вычисляем доходности за периоды 1, 24, 168, 720 часов
    periods = [1, 24, 168, 720]
    for period in periods:
        df[f'Return_{period}h'] = df['Close'].pct_change(periods=period)
    
    # Нормализуем доходности
    # Нормализация: Return_{period}h_normalized = Return_{period}h / (Volatility * sqrt(period))
    for period in periods:
        # Используем волатильность, смещенную на период назад, чтобы не использовать будущее значение
        df[f'Volatility_{period}h'] = df['Volatility'].shift(period)
        df[f'Return_{period}h_Normalized'] = df[f'Return_{period}h'] / (df[f'Volatility_{period}h'] * np.sqrt(period))
    
    macd_indicator = ta.trend.MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
    df['MACD'] = macd_indicator.macd()  # Линия MACD
    df['MACD_Signal'] = macd_indicator.macd_signal()  # Сигнальная линия MACD
    df['MACD_Hist'] = macd_indicator.macd_diff()  # Гистограмма MACD

    df['RSI'] = ta.momentum.rsi(df['Close'], window=14)

    df['Asset_ID_Encoded'] = valid_tokens[token_name]

    columns_to_keep = [
        # 'Date',
        'Close',
        'Close_Normalized',
        'Return_1h_Normalized',
        'Return_24h_Normalized',
        'Return_168h_Normalized',
        'Return_720h_Normalized',
        'Volatility_Normalized',
        'MACD',
        'MACD_Signal',
        'MACD_Hist',
        'RSI'
    ]

    for column in columns_to_keep:
        df[column].replace(0, np.nan, inplace=True)
        df[column].interpolate(method='linear', inplace=True)
        df[column].fillna(method='bfill', inplace=True)
        df[column].fillna(method='ffill', inplace=True)

    # Оставляем только нужные столбцы
    df = df[columns_to_keep + ['Asset_ID_Encoded']]

    final_data.append(df)
    
    # Сохраняем обработанный DataFrame в список
    # split_point = int(len(df)*split_cnt)
    # train_data.append(df.iloc[:split_point])
    # test_data.append(df.iloc[split_point:])

In [21]:
final_df = pd.concat(final_data, ignore_index= True)

In [ ]:
final_df.head()

In [ ]:
final_df.describe()

In [ ]:
final_df.info()

In [25]:
final_df.to_csv(Path["train_data"](3,2))

In [ ]:
df.columns

In [27]:
def create_time_window(data):
    max_length = min([len(df) for df in data])
    num_iterations = max_length - time_window + 1
    sequences = []
    # Перебираем все возможные сдвиги
    for i in range(num_iterations):
        # Список для хранения данных текущей итерации
        current_data = []
        
        # Перебираем каждый токен
        for df in data:
            # Извлекаем данные с индекса i до i + time_window
            window_data = df.iloc[i:i+time_window].copy()
            
            # Добавляем идентификатор токена, если его нет
            if 'Asset_ID_Encoded' not in window_data.columns:
                window_data['Asset_ID_Encoded'] = df['Asset_ID_Encoded'].iloc[0]
                print(f"df withoun ID {df['Asset_ID_Encoded'].iloc[0]}")
            
            # Добавляем данные в текущий список
            current_data.append(window_data)
        
        # Объединяем данные всех токенов в один DataFrame
        iteration_df = pd.concat(current_data, ignore_index=True)
        
        # Добавляем столбец с меткой времени или индексом итерации
        iteration_df['Iteration'] = i
        
        # Добавляем DataFrame текущей итерации в общий список
        sequences.append(iteration_df)
        
        # (Опционально) Освобождаем память
        del iteration_df, current_data, window_data
    
    return sequences

In [ ]:
train_seq = create_time_window(data = train_data)
train_df = pd.concat(train_seq, ignore_index=True)
print(len(train_df['Iteration'].unique()))
train_df.head()

In [12]:
train_df.to_csv(Path["train_data"](2,4))

In [ ]:
test_seq = create_time_window(data = test_data)
test_df = pd.concat(test_seq, ignore_index=True)
print(len(test_df['Iteration'].unique()))
test_df.head()

In [14]:
test_df.to_csv(Path["test_data"](2,4))

In [ ]:
iteration_0_df = test_df[test_df['Iteration'] == 10]
iteration_0_df